In [1]:
from mc2hlib.common import load_pdf
from mc2hlib import lh
from rabbit import io_tools
import numpy as np
import copy 
import re
import lhapdf
import pandas as pd

In [2]:
pdf_name = "NNPDF40_nnlo_hessian_pdfas"
Q = 100
max_nf = 5
photon = False

In [3]:
pdf, fl, xgrid = load_pdf(pdf_name, Q, max_nf, photon)

LHAPDF 6.5.4 loading all 53 PDFs in set NNPDF40_nnlo_hessian_pdfas
NNPDF40_nnlo_hessian_pdfas, version 1; 53 PDF members


In [4]:
fitresult = io_tools.get_fitresult("/scratch/submit/cms/kdlong/CombineStudies/Unblinded/WMass_eta_pt_charge_nnpdf40/fitresults.hdf5")
labels, pulls, constraints = io_tools.get_pulls_and_constraints(
    fitresult,
    keep_nuisances="pdf.*",
)

2026-01-12 10:33:20.704226: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-12 10:33:20.749595: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-12 10:33:24.923707: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
/home/submit/kdlong/mc2hessian/env/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):
2026-01-12 10:33:26.728993: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: 

In [5]:
# The first entry is the zero variation by accident
pulls = pulls[1:]

In [6]:
new_pdf, fl, xgrid = load_pdf(pdf_name, Q, max_nf, photon)

In [7]:
cov = fitresult["cov"].get()
var_names = np.array(cov.axes["parms_x"])

pattern = re.compile(r"pdf\d+")
vec_match = np.vectorize(lambda x: bool(pattern.search(x)))
pdf_mask = vec_match(var_names)
# Remove the first entry which is just the nominal
pdf_mask[np.argmax(pdf_mask)] = False

cov_pdf = cov.values()[np.ix_(pdf_mask, pdf_mask)]

In [8]:
eigv, V = np.linalg.eigh(cov_pdf)
K = V*np.sqrt(np.maximum(eigv, 0))
cov_pdf.shape

(50, 50)

In [9]:
headers, grids = lh.load_all_replicas(pdf, lhapdf.paths()[0] + "/" + pdf.pdf_name + "/" + pdf.pdf_name)

-> Reading replica from LHAPDF 52 

In [10]:
# Don't include alpha_s
matrix = lh.big_matrix(grids[:-2])

/home/submit/kdlong/mc2hessian/env/lib/python3.12/site-packages/mc2hlib/lh.py:103: FutureWarning: The behavior of pd.concat with len(keys) != len(objs) is deprecated. In a future version this will raise instead of truncating to the smaller of the two sequences
  X = pd.concat(gridlist[1:], axis=1,


In [11]:
central_grid = grids[0]+np.sum(pulls*matrix, axis=1)

In [12]:
result = matrix.dot(K).add(central_grid, axis=0)

In [15]:
base = lhapdf.paths()[0] + "/" + pdf.pdf_name + "/" + pdf.pdf_name
set_name = "NNPDF40_nnlo_hessian_pdfas_cmsmw"
folder = "/scratch/submit/cms/wmass/PostfitPDF"
name = "/".join([folder, set_name])

In [17]:
inn = open(base + ".info", 'r')
out = open(folder + "/" + set_name + ".info", 'w')
for l in inn.readlines():
    if l.find("SetDesc:") >= 0: out.write(f"SetDesc: \"{pdf.pdf_name} modified by CMS mW postfit covariance\"\n")
    else: out.write(l)
inn.close()
out.close()

In [18]:
lh.write_replica(0, name, b"PdfType: 'central'\nFormat: lhagrid1\n", central_grid)
for column in result.columns:
    header = b"PdfType: 'error'\nFormat: lhagrid1\n"
    lh.write_replica(column+1, name, header, result[column])